# Trade derivatives

## Derivatives Types
There are three derivatives types available for trading at OKX. 
- Futures
- Perpetual swaps
- Options
You can refer to [Derivatives Explained](https://www.okx.com/academy/en/bitcoin-derivatives-explained-futures-perpetual-swaps-and-options) for explanations and more information. 
We will use Perpetual swaps as examples in this tutorial.


## Import API modules
The following modules are available
- Trade
- BlockTrading
- Funding
- Account
- Convert
- Earning
- SubAccount
- MarketData
- PublicData
- TradingData
- Status
- NDBroker
- FDBroker

In [ ]:
import okx.Trade as Trade

## Fill in your API key details

In [ ]:
api_key = "xxxxx"
secret_key = "xxxxx"
passphrase = "xxxxxx"

## Get available funds

In [ ]:
import okx.Funding as Funding

flag = "1"  # live trading:0 , demo trading：1

fundingAPI = Funding.FundingAPI(api_key, secret_key, passphrase, False, flag)

result = fundingAPI.get_currencies()
print(result)

## Get market data
You can also replace instType with "FUTURES" or "OPTION" for your information.

In [ ]:
import okx.MarketData as MarketData

flag = "1"  # live trading: 0, demo trading: 1

marketDataAPI = MarketData.MarketAPI(flag = flag)

result = marketDataAPI.get_tickers(instType = "SWAP")
print(result)

# Prepare for trading
- Make sure you understand the basic trading rules. Please refer to [Basic Trading Rules](https://www.okx.com/support/hc/en-us/sections/360011507312)
- Make sure you have enough funds in your trading account. Please refer to  [Get balance](https://www.okx.com/docs-v5/en/#rest-api-account-get-balance).

## Get account balance. Please refer to  [Get balance](https://www.okx.com/docs-v5/en/#rest-api-account-get-balance).

In [ ]:
import okx.Account as Account
flag = "1"  # live trading: 0, demo trading: 1

accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)

result = accountAPI.get_account_balance()
print(result)

## Get available trading pairs from [Get instruments](https://www.okx.com/docs-v5/en/#rest-api-public-data-get-instruments).
As in the same manner, choose the instType that you would like to get information for.

In [ ]:
import okx.PublicData as PublicData

if __name__ == '__main__':

    flag = "1"  # live trading: 0, demo trading: 1

    publicDataAPI = PublicData.PublicAPI(flag = flag)

    result = publicDataAPI.get_instruments(instType = "SWAP")
    print(result)

## Get the current account configuration from the `acctLv` parameter in [Get account configuration](https://www.okx.com/docs-v5/en/#rest-api-account-get-account-configuration).
In unified account, there are four account modes as we mentioned in the last tutorial: 
- Simple account,
- Single-currency margin account, 
- Multi-currency margin account, 
- Portfolio margin account. 

Only the last three margin modes, namely, single-currency margin, multi-currency margin and portfolio margin, are eligible to trade derivatives.

In [ ]:
import okx.Account as Account

flag = "1"  # live trading: 0, demo trading: 1

accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)
result = accountAPI.get_account_config()
print(result)

if result['code'] == "0":
    acctLv = result["data"][0]["acctLv"]
    if acctLv == "1":
        print("Simple mode")
    elif acctLv == "2":
        print("Single-currency margin mode")
    elif acctLv == "3":
        print("Multi-currency margin mode")
    elif acctLv == "4":
        print("Portfolio margin mode")

## Set leverage via  [Set leverage](https://www.okx.com/docs-v5/en/#rest-api-account-set-leverage).
You can refer to [OKX Margin Trading Rules](https://www.okx.com/support/hc/en-us/articles/360019908352--OKX-Margin-Trading-Rules).
The leverage of a single currency, Leverage = position value / (balance in cross positions + unrealized P&L in cross margin positions).
See [references of setting leverage](https://www.okx.com/trade-market/position/swap) for more information.

In [ ]:
# Set leverage to be 5x for all cross-margin BTC-USDT SWAP positions,
# by providing any SWAP instId with BTC-USDT as the underlying
result = accountAPI.set_leverage(
    instId = "BTC-USDT-SWAP",
    lever = "5",
    mgnMode = "cross"
)
print(result)

# Set leverage to be 5x for all isolated-margin BTC-USDT SWAP positions,
# by providing any SWAP instId with BTC-USDT as the underlying
result = accountAPI.set_leverage(
    instId = "BTC-USDT-SWAP",
    lever = "5",
    mgnMode = "isolated"
)
print(result)

# Set leverage to be 5x for an isolated-margin 
# BTC-USDT-SWAP long position;
# This does NOT affect the leverage of any other BTC-USDT SWAP positions with different maturities or posSide
result = accountAPI.set_leverage(
    instId = "BTC-USDT-SWAP",
    lever = "5",
    posSide = "long",
    mgnMode = "isolated"
)
print(result)

# Start Derivatives Trading

### Derivatives trading under single-currency margin/multi-currency margin/portfolio margin mode

In [ ]:
import okx.Trade as Trade

flag = "1"  # live trading: 0, demo trading: 1

tradeAPI = Trade.TradeAPI(api_key, secret_key, passphrase, False, flag)

### Place orders under different position (order placement) modes: long/short and buy/sell
There are two position (order placement) modes when trading FUTURES and perpetual SWAP: long/short and buy/sell (net).
You can change the position (order placement) mode between long/short and buy/sell (net), via the API [Set position mode](https://www.okx.com/docs-v5/en/#rest-api-account-set-position-mode):

In [ ]:
result = accountAPI.set_position_mode(
    posMode="long_short_mode"
)
print(result)

### Place a limit order via [Place order](https://www.okx.com/docs-v5/en/#rest-api-trade-place-order)

In [ ]:
# limit order
result = tradeAPI.place_order(
    instId="BTC-USDT-SWAP",
    tdMode="isolated",
    side="buy",
    posSide="net",
    ordType="limit",
    px="19000",
    sz="100"
)
print(result)

if result["code"] == "0":
    print("Successful order request，order_id = ",result["data"][0]["ordId"])
else:
    print("Unsuccessful order request，error_code = ",result["data"][0]["sCode"], ", Error_message = ", result["data"][0]["sMsg"])

### Place a market order

In [ ]:
# market order
result = tradeAPI.place_order(
    instId="BTC-USDT-SWAP",
    tdMode="isolated",
    side="buy",
    posSide="net",
    ordType="market",
    sz="100"
)
print(result)

if result["code"] == "0":
    print("Successful order request，order_id = ",result["data"][0]["ordId"])
else:
    print("Unsuccessful order request，error_code = ",result["data"][0]["sCode"], ", Error_message = ", result["data"][0]["sMsg"])

## Cancel an order via [Cancel order](https://www.okx.com/docs-v5/en/#rest-api-trade-cancel-order)
You also use clOrdId in place of ordId

In [ ]:
result = tradeAPI.cancel_order(instId="BTC-USDT-SWAP", ordId="505073046126960640")
print(result)

## Amend an order via [Amend order](https://www.okx.com/docs-v5/en/#rest-api-trade-amend-order)
You also use clOrdId in place of ordId.
This example shows the revision of a new size.

In [ ]:
result = tradeAPI.amend_order(
    instId="BTC-USDT-SWAP", 
    ordId="505073046126960640",
    newSz="80"
)
print(result)

## Get details/state of a certain order, please refer to [Get order details](https://www.okx.com/docs-v5/en/#rest-api-trade-get-order-details)
Other than ordId, you can also specify clOrdId to get order details.

In [ ]:
result = tradeAPI.get_order(instId="BTC-USDT-SWAP", ordId="505073046126960640")
print(result)

## To get the list of open orders, please refer to [Get order List](https://www.okx.com/docs-v5/en/#rest-api-trade-get-order-list)

In [ ]:
result = tradeAPI.get_order_list()
print(result)

## To get past orders，please refer to [Get order history (last 7 days)](https://www.okx.com/docs-v5/en/#rest-api-trade-get-order-history-last-7-days) and [Get order history (last 3 months)](https://www.okx.com/docs-v5/en/#rest-api-trade-get-order-history-last-3-months)

In [ ]:
result = tradeAPI.get_orders_history(
    instType="SPOT"
)
print(result)

In [ ]:
result = tradeAPI.get_orders_history_archive(
    instType="SPOT"
)
print(result)

## To get past trades，please refer to [Get transaction details (last 3 days)](https://www.okx.com/docs-v5/en/#rest-api-trade-get-transaction-details-last-3-days) and [Get transaction details (last 3 months)](https://www.okx.com/docs-v5/en/#rest-api-trade-get-transaction-details-last-3-months)

In [ ]:
result = tradeAPI.get_fills(
    instType = "SWAP"
)
print(result)

In [ ]:
result = tradeAPI.get_fills_history(
    instType = "SWAP"
)
print(result)

## Get positions via [Get positions](https://www.okx.com/docs-v5/en/#rest-api-account-get-positions)
When your account is in net mode, net positions will be displayed, and when your account is in long/short mode, long or short positions will be displayed.

In [ ]:
result = accountAPI.get_positions()
print(result)

For example, you can track your unrealized profit and loss through the response parameter upl.